In [5]:
# Import dependencies
import pandas as pd
import requests
import numpy as np
from time import sleep
import json
import pretty-format-json as pp

SyntaxError: invalid syntax (1281447108.py, line 7)

In [2]:
# Read in CSV file
path = "Full-Dataset-(2018-2020).csv"
df = pd.read_csv(path)

# Remove un-needed columns
clean_df = df.drop(columns=["General.Project","Contact.Person.s.Last.Name","Timezone","Country","Camera.Make","Camera.Model","Flash..Yes.or.No.","Bait.Used","Photos.per.trigger","Description"])
clean_df.head()

#Filter to just Wasatch
clean_df = clean_df.loc[clean_df["General.Sampling.Period.Name"].str.contains("Wasatch Mountains"),:]
clean_df

clean_df['Date'] = pd.to_datetime(df['Date.Time']).dt.date
clean_df['Time'] = pd.to_datetime(df['Date.Time']).dt.time
clean_df

,Station,Wild.Urban,Longitude,Latitude,General.Sampling.Period.Name,Photo.Date,Photo.Time,Date.Time,Class,Order,Family,Genus,Species,Common.Name,Species.Diet,Number.of.Animals,Date,Time
0,d47256,Wild,-111.70713,40.80159,Wasatch Mountains (May - October 2018),5/6/18,17:21:06,2018-05-06 17:21:06,Mammalia,Primates,Hominidae,Homo,Sapiens,Human non-staff,Omnivore,1,2018-05-06,17:21:06
1,d47256,Wild,-111.70713,40.80159,Wasatch Mountains (May - October 2018),5/6/18,17:30:29,2018-05-06 17:30:29,Mammalia,Primates,Hominidae,Homo,Sapiens,Human non-staff,Omnivore,2,2018-05-06,17:30:29
2,d47256,Wild,-111.70713,40.80159,Wasatch Mountains (May - October 2018),5/6/18,17:36:20,2018-05-06 17:36:20,Mammalia,Primates,Hominidae,Homo,Sapiens,Human non-staff,Omnivore,2,2018-05-06,17:36:20
3,d47256,Wild,-111.70713,40.80159,Wasatch Mountains (May - October 2018),5/6/18,18:13:36,2018-05-06 18:13:36,Mammalia,Carnivora,Canidae,Canis,Lupus Familiaris,Domestic Dog,Carnivore,1,2018-05-06,18:13:36
4,d47256,Wild,-111.70713,40.80159,Wasatch Mountains (May - October 2018),5/6/18,18:13:36,2018-05-06 18:13:36,Mammalia,Primates,Hominidae,Homo,Sapiens,Human non-staff,Omnivore,2,2018-05-06,18:13:36
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93653,d71350,Urban,-111.81272,40.54350,Wasatch Mountains (April - November 2020),11/4/20,4:42:48,2020-11-04 04:42:48,Mammalia,Cetartiodactyla,Cervidae,Odocoileus,Hemionus,Mule Deer,Herbivore,1,2020-11-04,04:42:48
93732,d71226,Wild,-111.77295,40.79888,Wasatch Mountains (April - November 2020),11/6/20,6:04:46,2020-11-06 06:04:46,Mammalia,Carnivora,Mephitidae,Mephitis,Mephitis,Striped Skunk,Omnivore,1,2020-11-06,06:04:46
93733,d71226,Wild,-111.77295,40.79888,Wasatch Mountains (April - November 2020),11/6/20,6:08:59,2020-11-06 06:08:59,Mammalia,Carnivora,Mephitidae,Mephitis,Mephitis,Striped Skunk,Omnivore,1,2020-11-06,06:08:59
93734,d71226,Wild,-111.77295,40.79888,Wasatch Mountains (April - November 2020),11/6/20,20:59:13,2020-11-06 20:59:13,Mammalia,Carnivora,Mephitidae,Mephitis,Mephitis,Striped Skunk,Omnivore,1,2020-11-06,20:59:13


In [25]:
# Create new data with just the days of data collected for pulling weather API data
# This is super slow. Only run if needed.
date = clean_df["Date"].unique().astype(str)
date = date.tolist()

# Initiate empty lists 
weathercode = []
temperature_2m_max = []
temperature_2m_min = []
temperature_2m_mean = []
sunrise = []
sunset = []
precipitation_sum = []
rain_sum = []
snowfall_sum = []
date
lon = round(clean_df["Longitude"].mean(),2)
lat = round(clean_df["Latitude"].mean(),2)

# Create API pull and fill in the empty lists
for d in date:
    url = "https://archive-api.open-meteo.com/v1/archive?"
    weather = url + "latitude=" + str(lat) + "&longitude=" + str(lon) + "&start_date=" + d + "&end_date=" + d + "&daily=weathercode,temperature_2m_max,temperature_2m_min,temperature_2m_mean,sunrise,sunset,precipitation_sum,rain_sum,snowfall_sum&temperature_unit=fahrenheit&windspeed_unit=mph&precipitation_unit=inch&timezone=America%2FDenver"
    weather_json = requests.get(weather).json()
    weathercode.append(weather_json["daily"]["weathercode"])
    temperature_2m_max.append(weather_json["daily"]["temperature_2m_max"])
    temperature_2m_min.append(weather_json["daily"]["temperature_2m_min"])
    temperature_2m_mean.append(weather_json["daily"]["temperature_2m_mean"])
    sunrise.append(weather_json["daily"]["sunrise"])
    sunset.append(weather_json["daily"]["sunset"])
    precipitation_sum.append(weather_json["daily"]["precipitation_sum"])
    rain_sum.append(weather_json["daily"]["rain_sum"])
    snowfall_sum.append(weather_json["daily"]["snowfall_sum"])

In [33]:
# Create new data frame with API data
weather_data = {"Date":date,
                "Weathercode":weathercode,
                "Max Temperature":temperature_2m_max,
                "Min Temperature":temperature_2m_min,
                "Mean Temperature":temperature_2m_mean,
                "Sunrise":sunrise,
                "Sunset":sunset,
                "Total Precipitation":precipitation_sum,
                "Total Rain":rain_sum,
                "Total Snow":snowfall_sum}

weather_df = pd.DataFrame(weather_data)
weather_df

# create csv
path2 = "WeatherAPIData.csv"
weather_df.to_csv(path2)

In [30]:
merged_df = pd.merge(clean_df,weather_df,on="Date",how="left")
merged_df

,Station,Wild.Urban,Longitude,Latitude,General.Sampling.Period.Name,Photo.Date,Photo.Time,Date.Time,Class,Order,...,Time,Weathercode,Max Temperature,Min Temperature,Mean Temperature,Sunrise,Sunset,Total Precipitation,Total Rain,Total Snow
0,d47256,Wild,-111.70713,40.80159,Wasatch Mountains (May - October 2018),5/6/18,17:21:06,2018-05-06 17:21:06,Mammalia,Primates,...,17:21:06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,d47256,Wild,-111.70713,40.80159,Wasatch Mountains (May - October 2018),5/6/18,17:30:29,2018-05-06 17:30:29,Mammalia,Primates,...,17:30:29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,d47256,Wild,-111.70713,40.80159,Wasatch Mountains (May - October 2018),5/6/18,17:36:20,2018-05-06 17:36:20,Mammalia,Primates,...,17:36:20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,d47256,Wild,-111.70713,40.80159,Wasatch Mountains (May - October 2018),5/6/18,18:13:36,2018-05-06 18:13:36,Mammalia,Carnivora,...,18:13:36,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,d47256,Wild,-111.70713,40.80159,Wasatch Mountains (May - October 2018),5/6/18,18:13:36,2018-05-06 18:13:36,Mammalia,Primates,...,18:13:36,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53838,d71350,Urban,-111.81272,40.54350,Wasatch Mountains (April - November 2020),11/4/20,4:42:48,2020-11-04 04:42:48,Mammalia,Cetartiodactyla,...,04:42:48,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
53839,d71226,Wild,-111.77295,40.79888,Wasatch Mountains (April - November 2020),11/6/20,6:04:46,2020-11-06 06:04:46,Mammalia,Carnivora,...,06:04:46,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
53840,d71226,Wild,-111.77295,40.79888,Wasatch Mountains (April - November 2020),11/6/20,6:08:59,2020-11-06 06:08:59,Mammalia,Carnivora,...,06:08:59,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
53841,d71226,Wild,-111.77295,40.79888,Wasatch Mountains (April - November 2020),11/6/20,20:59:13,2020-11-06 20:59:13,Mammalia,Carnivora,...,20:59:13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
